In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
import utils, numpy as np
from tqdm import tqdm

X, y    = utils.load_data()
prep    = utils.make_preprocessor(X.columns.tolist())
cv      = utils.get_cv()
TARGETS = utils.TARGETS

def make_pipe(base):
    return Pipeline([("prep", prep), ("rf", base)])

print("▼ Single-task RF  (20 iter RandomizedSearch)")
for t in tqdm(TARGETS):
    base = RandomForestRegressor(random_state=0, n_jobs=-1)
    pipe = make_pipe(base)
    best, *_ = utils.tune_model(pipe, X, y[t], cv)
    rmse, r2 = utils.cv_metrics(best, X, y[t], cv)
    print(f"{t:<12}  RMSE={rmse:6.2f}   R²={r2:6.3f}")

print("\n▼ Multi-task RF")
base_mt = MultiOutputRegressor(RandomForestRegressor(random_state=0, n_jobs=-1), n_jobs=-1)
pipe_mt, *_ = utils.tune_model(make_pipe(base_mt), X, y, cv)
rmse_mt, r2_mt = utils.cv_metrics(pipe_mt, X, y, cv)
print(f"平均 RMSE = {rmse_mt:6.2f}   平均 R² = {r2_mt:6.3f}")

In [ ]:
import utils, importlib, inspect, os
print("✔  utils is:", utils.__file__)           # どのパスの utils を読んでいる？
print("✔  tune_model source↓")
print(inspect.getsource(utils.tune_model)[:400])  # 冒頭だけ表示

In [ ]:
# デバッグ用パイプラインを1本だけ作る
dummy_pipe = Pipeline([
    ("prep", utils.make_preprocessor([])),        # 入力列は空でもよい
    ("rf"  , RandomForestRegressor())
])

print("🔍 param_dist keys ↓")
print(utils.rf_param_dist(dummy_pipe).keys())


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import utils

dummy_pipe = Pipeline([
    ("prep", utils.make_preprocessor([])),
    ("rf",   RandomForestRegressor())
])
print(utils.rf_param_dist(dummy_pipe).keys())